# Beta Bank

El banco Beta Bank ha identificado que sus clientes se están yendo y consideraron que lo mas óptimo es lograr convencer a los aún clientes de permanecer en el banco.

Para ese objetivo se nos ha proporcionado data sobre el comportamiento pasado de los clientes y la terminación de contratos.
Lo que se pretende obtener son las predicciones de si un cliente dejará el banco o no.

Necesitamos para este requerimiento un modelo de clasificación binaria que seleccione 1 si el cliente va a dejar el banco y 0 en caso negativo.

Mediremos el éxito del modelo basado en la métrica F1, compararemos la matriz de confusión junto con la precisión, el recall y el AUC-ROC.

Previo a la modelación es necesario preprocesar los datos, validar duplicados, quitar valores ausentes que puedan alterar el modelo, codificar las columnas necesarias, estandarizar los valores de cada columna numérica, esto último dependiendo del modelo elegido.

## Importación de librerías, descarga y preprocesamiento de los datos

### Importación y lectura de datos

In [18]:
# importar librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, recall_score, precision_score, precision_recall_curve, f1_score, confusion_matrix, roc_curve, roc_auc_score
pd.set_option('mode.chained_assignment', None)

In [19]:
# leer datos
data = pd.read_csv('./Churn.csv')

In [20]:
data.sample()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
3837,3838,15662533,Porter,598,Spain,Female,23,6.0,0.0,2,1,0,153229.19,0


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [22]:
100 * data.Tenure.isna().sum() / data.shape[0]

9.09

La columna `Tenure` tiene valores ausentes, 9.09% de sus registros no tienen información y dado que representa los años que un cliente ha permanecido en el banco es importante que en la columna no haya valores `nan`.

Hemos encontrado la misma información en kaggle, así que usaremos ese dataset.

In [23]:
data = pd.read_csv('./Churn_Modelling.csv')

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


Vamos a estandarizar las columnas al formato snake-case y veremos los valores únicos de las columnas categóricas.

In [25]:
data.columns = data.columns.str.lower()
data.columns = ['row_number', 'customer_id', 'surname', 'credit_score', 'geography',
                'gender', 'age', 'tenure', 'balance', 'num_of_products', 'has_cr_card',
                'is_active_member', 'estimated_salary', 'exited']

In [26]:
list = ['geography',
        'gender', 'tenure', 'num_of_products', 'has_cr_card',
        'is_active_member', 'exited']
for column in list:
    print(column, data[column].sort_values().unique())

geography ['France' 'Germany' 'Spain']
gender ['Female' 'Male']
tenure [ 0  1  2  3  4  5  6  7  8  9 10]
num_of_products [1 2 3 4]
has_cr_card [0 1]
is_active_member [0 1]
exited [0 1]


In [27]:
data.duplicated().sum()

0

No hay ningún duplicado por registro. El tipo de datos de cada columna es la correcta.

Las primeras tres columnas no proveen algun tipo de información relevante para el modelo, ya se es el número de fila, el id de cliente y el apellido, lo que optaremos por quitarlos del dataset.

In [28]:
data = data.drop(['row_number', 'customer_id', 'surname'], axis=1)

### Codificación de columnas categóricas

Se pasará a codificar la data de acuerdo a los modelos que se van a probar: regresión logística, árboles y bosques.

Para la Regresión Logística se codificará con OHE. Y para los árboles y bosques por codificación de etiquetas.

In [29]:
data_ohe = pd.get_dummies(data, drop_first=True)

In [30]:
data_ohe.sample()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited,geography_Germany,geography_Spain,gender_Male
5525,624,51,10,123401.43,2,1,1,127825.25,0,False,False,True


In [31]:
encoder = OrdinalEncoder()
data_ordinal = pd.DataFrame(encoder.fit_transform(data), columns=data.columns)

In [32]:
data_ordinal.sample()

,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
1925,413.0,0.0,1.0,6.0,3.0,0.0,1.0,1.0,0.0,8648.0,0.0


### Estandarización de columnas numéricas y segmentación de conjuntos

Debido a la gran diferencia en valores de las columnas numéricas vamos a estandarizar las columnas, restando la media y dividiendo por la desviasión estandar. Esto después de la división de los datos de entrenamiento, validación y pruebas para el dataset para regresión lineal y para los modelos de árboles.

Dado que tenemos solo este dataset, lo dividiremos en proporcion 3:1:1

In [33]:
# segmentar por características y objetivo
target_ohe = data_ohe['exited']
features_ohe = data_ohe.drop('exited', axis=1)

# segmentar por conjunto de datos
# 60 train : 20 valid : 20 test
features_train_ohe, features_rest_ohe, target_train_ohe, target_rest_ohe = train_test_split(
    features_ohe, target_ohe, test_size=0.40, random_state=12345)

features_valid_ohe, features_test_ohe, target_valid_ohe, target_test_ohe = train_test_split(
    features_rest_ohe, target_rest_ohe, test_size=0.50, random_state=12345)

In [34]:
numeric = ['credit_score', 'age', 'tenure',
           'balance', 'num_of_products', 'estimated_salary']

In [35]:
# estandarizar las variables numericas
scaler = StandardScaler()
scaler.fit(features_train_ohe[numeric])
features_train_ohe[numeric] = scaler.transform(features_train_ohe[numeric])
features_valid_ohe[numeric] = scaler.transform(features_valid_ohe[numeric])
features_test_ohe[numeric] = scaler.transform(features_test_ohe[numeric])

Ahora para el dataset de los modelos de árboles.

In [36]:
# segmentar por características y objetivo
target_ordinal = data_ordinal['exited']
features_ordinal = data_ordinal.drop('exited', axis=1)

# segmentar por conjunto de datos
# 60 train : 20 valid : 20 test
features_train_ordinal, features_rest_ordinal, target_train_ordinal, target_rest_ordinal = train_test_split(
    features_ordinal, target_ordinal, test_size=0.40, random_state=12345)

features_valid_ordinal, features_test_ordinal, target_valid_ordinal, target_test_ordinal = train_test_split(
    features_rest_ordinal, target_rest_ordinal, test_size=0.50, random_state=12345)

In [ ]:
# estandarizar las variables numericas
scaler = StandardScaler()
scaler.fit(features_train_ordinal[numeric])
features_train_ordinal[numeric] = scaler.transform(
    features_train_ordinal[numeric])
features_valid_ordinal[numeric] = scaler.transform(
    features_valid_ordinal[numeric])
features_test_ordinal[numeric] = scaler.transform(
    features_test_ordinal[numeric])

## Modelización de datos

### Evaluación de equilibrio en la variable objetivo

Ya que está listo el dataset para crear los modelos veamos si la variable objetivo esta balanceada.

In [38]:
100 * data_ohe['exited'].value_counts() / data_ohe.shape[0]

exited
0    79.63
1    20.37
Name: count, dtype: float64

Por la proporción podemos ver que casi el 80% de los clientes siguen con el banco, y el 20% han dejado al banco.
Estamos ante un desequilibrio de clase de la variable objetivo.

Probemos como predice un modelo de un árbol de decisión, se evaluará con la métrica de exactitud.

### Modelización con desequilibrio de la variable objetivo

In [39]:
# modelo desbalanceado
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_train_ordinal, target_train_ordinal)

predict = model.predict(features_valid_ordinal)

accuracy = accuracy_score(target_valid_ordinal, predict)
f1 = f1_score(target_valid_ordinal, predict)

print('Exactitud de la predición del modelo de árbol de decisión:', accuracy)
print('F1', f1)

Exactitud de la predición del modelo de árbol de decisión: 0.772
F1 0.46352941176470586


El modelo tiene una exactitud del 77.2% es decir 77.2% de sus predicciones fueron reales.

Realizemos una prueba de cordura para ver si un modelo constante puede predecir mejor que el modelo, si el modelo constante arroja mejor exactitud nuestro modelo de árbol de decisión es obsoleto y requerirá otro tipo de tratamiento para mejorar la métrica.

### Prueba de cordura

In [40]:
# modelo constante
pred_constant = pd.Series(0, index=features_valid_ordinal.index)

accuracy = accuracy_score(target_valid_ordinal, pred_constant)
f1 = f1_score(target_valid_ordinal, pred_constant)

print('Exactitud de la predicción del modelo constante:', accuracy)
print('F1', f1)

Exactitud de la predicción del modelo constante: 0.791
F1 0.0


El modelo constante obtuvo mayor exactitud, por lo que nuestro modelo requiere ajustes a ese desequilibrio identificado.

### Modelización con el algoritmo de regresión logística balanceada

In [41]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train_ohe, target_train_ohe)

probabilities_valid = model.predict_proba(features_valid_ohe)
probabilities_one_valid = probabilities_valid[:, 1]

best_f1 = 0
best_threshold = 0
best_auc_roc = 0
best_cm = 0

for threshold in np.arange(0, 0.3, 0.02):
    predict = probabilities_one_valid > threshold

    f1 = f1_score(target_valid_ohe, predict)
    auc_roc = roc_auc_score(target_valid_ohe, predict)
    cm = confusion_matrix(target_valid_ohe, predict)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold
        best_auc_roc = auc_roc
        best_cm = cm

lr = [best_threshold, best_f1, best_auc_roc]

print('best_threshold', best_threshold)
print('best_f1:', best_f1)
print('best_auc_roc', best_auc_roc)
print(best_cm)

best_threshold 0.26
best_f1: 0.505091649694501
best_auc_roc 0.6967771399536653
[[1266  316]
 [ 170  248]]


El modelo logró un F1 de `0.505091649694501` (mayor al del modelo desbalanceado de árbol de decisión: `0.4635294117647059`) con un umbral del `0.26`, sin embargo nos han pedido un modelo con el F1 mayor a 0.59.

Vamos a crear un modelo con árbol de decisión con los datos balanceados.

### Modelización con el algoritmo de árbol de decisión balanceada

In [42]:
best_depth = 0
best_threshold = 0
best_f1 = 0
best_auc_roc = 0
best_cm = 0

for depth in range(1, 46):
    # inicializa el constructor de modelos con los parámetros random_state=12345 y max_depth=depth
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    # entrena el modelo en el conjunto de entrenamiento
    model.fit(features_train_ordinal, target_train_ordinal)

    # obtén las predicciones del modelo en el conjunto de validación
    probabilities_valid = model.predict_proba(features_valid_ordinal)
    probabilities_one_valid = probabilities_valid[:, 1]

    for threshold in np.arange(0, 0.3, 0.02):
        predict = probabilities_one_valid > threshold

        f1 = f1_score(target_valid_ordinal, predict)
        auc_roc = roc_auc_score(target_valid_ordinal, predict)
        cm = confusion_matrix(target_valid_ordinal, predict)
        if f1 > best_f1:
            best_depth = depth
            best_threshold = threshold
            best_f1 = f1
            best_auc_roc = auc_roc
            best_cm = cm
dtc = [best_threshold, best_f1, best_auc_roc]

print("F1 del mejor modelo de árbol de decisión en el conjunto de validación:",
      best_f1, ',best_depth:', best_depth, ',best_threshold', best_threshold)
print("El AUC_ROC del mejor modelo evaluado por F1:", best_auc_roc)
print(best_cm)

F1 del mejor modelo de árbol de decisión en el conjunto de validación: 0.6082949308755761 ,best_depth: 5 ,best_threshold 0.26
El AUC_ROC del mejor modelo evaluado por F1: 0.7570031272872446
[[1396  186]
 [ 154  264]]


El modelo de árbol de decisión que mejor métrica F1 obtuvo fue con una profundidad de 5 y un umbral de `0.26` con valor de: `0.608294930875576`.

Evaluemos el modelo de bosque aleatorio con una nueva técnica de balanceo.

### Modelización con el algoritmo de bosque aleatorio balanceada

Primero veamos cuanta escala tiene la variable objetivo. Y cual es la fracción de la clase positiva frente a la clase negativa.

In [ ]:
repeat = target_train_ohe.value_counts(
)[0] / target_train_ohe.value_counts()[1]


repeat

4.016722408026756

In [ ]:
fraction = target_train_ohe.value_counts(
)[1] / target_train_ohe.value_counts()[0]


fraction

0.24895920066611157

In [45]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    return features_upsampled, target_upsampled

In [46]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat([features_zeros.sample(
        frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(
        frac=fraction, random_state=12345)] + [target_ones])

    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)

    return features_downsampled, target_downsampled

Optaremos por el sobremuestreo para el modelo de bosque aleatorio.

In [ ]:
features_upsampled, target_upsampled = upsample(
    features_train_ordinal, target_train_ordinal, round(repeat))



best_est = 0


best_depth = 0


best_f1 = 0


best_auc_roc = 0


best_cm = 0


for est in range(10, 51, 10):


    for depth in range(1, 11):


        # inicializa el constructor de modelos con los parámetros random_state=12345, n_estimators=est y max_depth=depth


        model = RandomForestClassifier(
            random_state=12345, n_estimators=est, max_depth=depth)


        # entrena el modelo en el conjunto de entrenamiento


        model.fit(features_upsampled, target_upsampled)


        # obtén las predicciones del modelo en el conjunto de validación


        predic = model.predict(features_valid_ordinal)


        probabilities_valid = model.predict_proba(features_valid_ordinal)


        probabilities_one_valid = probabilities_valid[:, 1]


        # calcula F1 y auc_roc en el conjunto de validación


        f1 = f1_score(target_valid_ordinal, predict)


        auc_roc = roc_auc_score(target_valid_ordinal, predic)


        cm = confusion_matrix(target_valid_ordinal, predict)


        if f1 > best_f1:
            best_f1 = f1
            best_auc_roc = auc_roc
            best_est = est


            best_depth = depth
            best_cm = cm



print("F1 del mejor modelo de bosque aleatorio en el conjunto de validación:",
      best_f1, "n_estimators:", best_est, "best_depth:", best_depth)


print("El AUC_ROC del mejor modelo evaluado por F1:", best_auc_roc)


print(best_cm)

F1 del mejor modelo de bosque aleatorio en el conjunto de validación: 0.46352941176470586 n_estimators: 10 best_depth: 1
El AUC_ROC del mejor modelo evaluado por F1: 0.733061535576673
[[1347  235]
 [ 221  197]]


Parece que este balanceo no resultó la mejor opción, la métrica F1 bajo considerablemente.
Cambiemos el método de balanceo al de umbral.

In [48]:
best_est = 0
best_depth = 0
best_threshold = 0
best_f1 = 0
best_auc_roc = 0
best_cm = 0
for est in range(10, 51, 10):
    for depth in range(1, 11):
        # inicializa el constructor de modelos con los parámetros random_state=12345, n_estimators=est y max_depth=depth
        model = RandomForestClassifier(
            random_state=12345, n_estimators=est, max_depth=depth)

        # entrena el modelo en el conjunto de entrenamiento
        model.fit(features_train_ordinal, target_train_ordinal)

        # obtén las predicciones del modelo en el conjunto de validación
        probabilities_valid = model.predict_proba(features_valid_ordinal)
        probabilities_one_valid = probabilities_valid[:, 1]

        for threshold in np.arange(0, 0.3, 0.02):
            predict = probabilities_one_valid > threshold

            f1 = f1_score(target_valid_ordinal, predict)
            auc_roc = roc_auc_score(target_valid_ordinal, predict)
            cm = confusion_matrix(target_valid_ordinal, predict)

            if f1 > best_f1:
                best_est = est
                best_depth = depth
                best_threshold = threshold
                best_f1 = f1
                best_auc_roc = auc_roc
                best_cm = cm
rfc = [best_threshold, best_f1, best_auc_roc]

print('F1 del mejor modelo de bosque aleatorio en el conjunto de validación:', best_f1,
      "n_estimators:", best_est, "best_depth:", best_depth, 'best_threshold', best_threshold)
print("El AUC_ROC del mejor modelo evaluado por F1:", best_auc_roc)
print(best_cm)

F1 del mejor modelo de bosque aleatorio en el conjunto de validación: 0.6334745762711864 n_estimators: 20 best_depth: 8 best_threshold 0.24
El AUC_ROC del mejor modelo evaluado por F1: 0.7859108753379829
[[1355  227]
 [ 119  299]]


La métrica F1 ha aumentado significativamente con cambiar el método de balanceo, se logró obtener un F1 de `0.6334745762711864` con un modelo de 20 árboles, 8 nodos de profundidad y un umbral de `0.24`.

### Resumen de los mejores modelos

Veamos las métricas de los 3 mejores modelos.

In [ ]:
metricas = {'regresión logística': lr,
            'árbol de decisión': dtc, 'bosque aleatorio': rfc}


metricas = pd.DataFrame(metricas, index=["umbral", "F1", "AUC_ROC"])


metricas

,regresión logística,árbol de decisión,bosque aleatorio
umbral,0.260000,0.260000,0.240000
F1,0.505092,0.608295,0.633475
AUC_ROC,0.696777,0.757003,0.785911


Los tres modelos arrojaron cifras superiores a si consideraramos un modelo aleatorio de `AUC_ROC = 0.50`, el modelo de bosque aleatorio se posiciona como el mejor, Evaluando con `F1`, la regresión logística dio un `F1 = 0.505092`, el árbol de decisión un `F1 = 0.608295` y el bosque aleatorio un `F1 = 0.633475`, nuevamente el modelo de bosque aleatorio se perfila como el mejor de los 3.

### Selección del mejor modelo

Hemos descubierto que el factor que aumentó las métricas es el umbral de clasificación y el modelo que ajustó la mayoría de los datos fue el bosque aleatorio con un F1 de: `0.633475` y un AUC_ROC de: `0.785911` superiores a los demás modelos.

Teniendo esto en consideración haremos la prueba final con el conjunto reservado para el mejor modelo.

### Prueba final para el modelo seleccionado

In [50]:
est = 20
depth = 8
threshold = 0.24

model = RandomForestClassifier(
    random_state=12345, n_estimators=est, max_depth=depth)

model.fit(features_train_ordinal, target_train_ordinal)

probabilities_valid = model.predict_proba(features_test_ordinal)
probabilities_one_valid = probabilities_valid[:, 1]

predict = probabilities_one_valid > threshold

f1 = f1_score(target_test_ordinal, predict)
auc_roc = roc_auc_score(target_test_ordinal, predict)
cm = confusion_matrix(target_test_ordinal, predict)
print('F1: ', f1, 'AUC_ROC: ', auc_roc)

F1:  0.601025641025641 AUC_ROC:  0.7642177519334524


Si bien las métricas no superaron los resultados con el conjunto de validación pero sí superaron la restricción inicial: F1 mayor o igual a `0.59`.

El modelo resultó predecir suficientemente bien los datos de prueba.

## Conclusión

Resumiendo los hallazgos, con el modelo seleccionado podemos hacer que Beta Bank implemente alguna estrategia de marketing que ofrezca promociones, descuentos, productos elite para los usuarios que el modelo haya predicho como positivos para dejar el banco.

Estas promociones no siempre llegarán con certeza a todos los usuarios que realmente dejarán el banco pero aseguramos que una gran parte caerá dentro de la zona positiva.

Dado que los banqueros decidieron que es menos costoso salvar a los clientes existentes optamos por compartir el modelo definitivo.